In [ ]:
from tools import unique_funcs, flatten_funcs, random_msg, chat_completion_request
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
msg = random_msg()
resp, latency = chat_completion_request([msg], tools=unique_funcs(1))
assert resp.status_code == 200
content = resp.json()['choices'][0]['message']['content']
if content: assert len(encoding.encode(content)) <= 15

In [ ]:
resp.json()

{'id': 'chatcmpl-8WLOGmmFf0vvoPm4uFvfi5GQWxeoV',
 'object': 'chat.completion',
 'created': 1702719412,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Of course! I can help you plan a road trip itinerary that includes'},
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 1071,
  'completion_tokens': 15,
  'total_tokens': 1086},
 'system_fingerprint': None}

In [ ]:
resp, latency = chat_completion_request([msg, flatten_funcs(unique_funcs(2))])
resp.json()

{'id': 'chatcmpl-8WLOKLs7sGZAbukgrfY3JhpHiKsKa',
 'object': 'chat.completion',
 'created': 1702719416,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Here's a road trip itinerary that includes landmarks and national parks across the United"},
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 723, 'completion_tokens': 15, 'total_tokens': 738},
 'system_fingerprint': None}

In [ ]:
resp, latency = chat_completion_request([msg])
resp.json()

{'id': 'chatcmpl-8WLOM3Y2JW6dWWe48IYVBgTXcxIkH',
 'object': 'chat.completion',
 'created': 1702719418,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Of course! Planning a road trip across the United States to visit landmarks and'},
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 37, 'completion_tokens': 15, 'total_tokens': 52},
 'system_fingerprint': None}

In [ ]:
def bench(n_tries=10):
    msg = random_msg()
    results = []
    for func_loc in ['System Message', 'Tools', None]:
        for i in range(n_tries):
            messages = [msg]
            func_tools = unique_funcs(i) if func_loc is not None else None

            if func_loc == 'System Message': 
                messages.append(flatten_funcs(func_tools))
                func_tools = None

            resp, latency = chat_completion_request(messages, tools=func_tools)
            
            results.append({'func_loc': func_loc, 'latency': latency, 'response':resp.json()})
            print(f"Func-Location: {func_loc} Iteration: {i} Latency: {latency:.2f}")
    return results

In [ ]:
bench_results = bench()

Func-Location: System Message Iteration: 0 Latency: 0.92
Func-Location: System Message Iteration: 1 Latency: 0.96
Func-Location: System Message Iteration: 2 Latency: 0.98
Func-Location: System Message Iteration: 3 Latency: 0.91
Func-Location: System Message Iteration: 4 Latency: 0.93
Func-Location: System Message Iteration: 5 Latency: 0.86
Func-Location: System Message Iteration: 6 Latency: 0.89
Func-Location: System Message Iteration: 7 Latency: 0.90
Func-Location: System Message Iteration: 8 Latency: 0.76
Func-Location: System Message Iteration: 9 Latency: 0.88
Func-Location: Tools Iteration: 0 Latency: 0.89
Func-Location: Tools Iteration: 1 Latency: 1.21
Func-Location: Tools Iteration: 2 Latency: 0.89
Func-Location: Tools Iteration: 3 Latency: 1.16
Func-Location: Tools Iteration: 4 Latency: 0.89
Func-Location: Tools Iteration: 5 Latency: 0.85
Func-Location: Tools Iteration: 6 Latency: 1.12
Func-Location: Tools Iteration: 7 Latency: 0.94
Func-Location: Tools Iteration: 8 Latency: 0.9

In [ ]:
bench_results[-1]

{'func_loc': None,
 'latency': 0.8525130748748779,
 'response': {'id': 'chatcmpl-8WLOqG9paOug1utknFZTERyc76LWm',
  'object': 'chat.completion',
  'created': 1702719448,
  'model': 'gpt-3.5-turbo-0613',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': 'Absolutely! Here are a few mystery novels that strike a balance between being engaging'},
    'logprobs': None,
    'finish_reason': 'length'}],
  'usage': {'prompt_tokens': 36, 'completion_tokens': 15, 'total_tokens': 51},
  'system_fingerprint': None}}

In [ ]:
bench_results[0]

{'func_loc': 'System Message',
 'latency': 0.9196739196777344,
 'response': {'id': 'chatcmpl-8WLOPxKCHidmgnVOEqQukn5p8pPhQ',
  'object': 'chat.completion',
  'created': 1702719421,
  'model': 'gpt-3.5-turbo-0613',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': 'Sure! Here are some mystery novels that are engaging but not too dark:\n\n'},
    'logprobs': None,
    'finish_reason': 'length'}],
  'usage': {'prompt_tokens': 722,
   'completion_tokens': 15,
   'total_tokens': 737},
  'system_fingerprint': None}}